# Задание по курсу «Дискретная оптимизация», МФТИ, весна 2017

## Задача 2-2. Эвристика Кернигана—Лина

В этой задаче Вам предлагается добавить к локальному поиска в задаче о сбалансированном разбиении графа эвристику Кернигана—Лина, когда мы, «застряв» в локальном минимуме, тем не менее пытаемся сделать несколько шагов из него, даже если они приводят к временному ухудшению. Надежда здесь на то, что после ухудшения может наступить заметное улучшение результата: нам удастся выпрыгнуть из локального оптимума. Мы рассматриваем безвесовый вариант задачи о разбиении с параметром балансировки $\alpha=\frac{1}{2}$:

**Даны:**
* $G=(V,E)$ — граф без весов на рёбрах

**Найти:**
* Разбиение $V=V'\sqcup V''$, такое, что $V'=\lfloor |V|/2 \rfloor$ и число рёбер между $V'$ и $V''$ минимально возможное.

Сделайте следующее:
* Скачайте файл [`partition-instances.zip`](https://github.com/dainiak/discrete-optimization-course/raw/master/partition-instances.zip) и разархивируйте из него файлы со входами задачи.
* Для каждого из графов найдите локальным поиском с эвристикой Кернигана—Лина локально минимальное (по количеству рёбер между частями) разбиение вершин графа на две части, мощности которых отличаются не более чем на единицу. 
* Реализуйте функцию `variable_depth_local_search`; она должна принимать на вход граф в формате, предоставляемом функцией `read_instance`, и возвращать найденное разбиение как множество вершин, лежащих в одной любой из двух компонент разбиения. Ваш локальный поиск должен начинаться с того разбиение, которое уже находится в переменной `starting_point`.
* Подберите для каждого из четырёх входных графов глубину поиска так, чтобы он работал не более 60 секунд на Вашем компьютере, и сохраните информацию о подобранных параметрах и любые свои интересные наблюдения в отдельную ячейку настоящего ipynb-файла.

In [2]:
def read_instance(filename):
    with open(filename, 'r') as file:
        n_vertices = int(file.readline().strip().split()[0])
        vertices, edges = set(range(1, n_vertices + 1)), set()
        for u in range(1, n_vertices + 1):
            for v in map(int, file.readline().strip().split()):
                edges.add((u,v))
        return (vertices, edges)

In [107]:
import random

num = [2900000, 2900000, 3000000, 90000]
numoper = 0

def variable_depth_local_search(graph):
    global numoper
    n = len(graph[0])
    starting_point = list(range(1, len(graph[0]) // 2 + 1))
    other = list(set(graph[0]) - set(starting_point))
    part = [0] * (n + 1)
    
    for v in other:
        part[v] = 1
    
    constli = 30
    constcnt = 2000
    
    ed = []
    
    for i in range(n + 1):
        ed.append([])
    
    sum = 0
    
    for e in graph[1]:
        ed[e[0]].append(e[1])
    
    cur_ans = 0
    
    for v in other:
        for u in ed[v]:
            if (part[u] == 0):
                cur_ans += 1
    
    k = len(starting_point)
    k1 = len(other)
    
    bad = 0
    curli = 0
    
    best = cur_ans
    
    answer = [0] * k
    
    for i in range(k):
        answer[i] = starting_point[i]
    
    for it in range(num[numoper]):
        posv = random.randint(0, k - 1)
        posu = random.randint(0, k1 - 1)
        v = starting_point[posv]
        u = other[posu]
        
        ch = 0
        
        for e in ed[v]:
            if (e == u):
                continue
            
            if (part[e] == 1):
                ch -= 1
            else:
                ch += 1
        
        for e in ed[u]:
            if (e == v):
                continue
            
            if (part[e] == 1):
                ch += 1
            else:
                ch -= 1
        
        if (ch < 0 or curli != 0):
            if (curli != 0):
                curli -= 1
                
            if (ch < 0):
                curli = 0
            
            starting_point[posv], other[posu] = other[posu], starting_point[posv]
            part[v] = 1
            part[u] = 0
            cur_ans += ch
            bad = 0
        else:
            bad += 1
            
            if (bad == constcnt):
                bad = 0
                curli = constli
        
        if (cur_ans < best):
            best = cur_ans
            
            for i in range(k):
                answer[i] = starting_point[i]
                
    numoper += 1
    return set(answer)

In [23]:
import time

def get_quality(graph, partition_part):
    if not (partition_part <= graph[0]) or abs(len(partition_part) - len(graph[0]) / 2) > 0.6:
        return -1
    other_part = set(graph[0]) - partition_part
    return sum(1 for edge in graph[1] if set(edge) <= partition_part or set(edge) <= other_part )

def run_all():
    filenames = ['add20.graph', 'cti.graph', 't60k.graph', 'm14b.graph']
    for filename in filenames:
        instance = read_instance(filename)
        print('Solving instance {}…'.format(filename), end='')
        time_start = time.monotonic()
        quality = get_quality(instance, variable_depth_local_search(instance))
        time_elapsed = time.monotonic()-time_start
        print(' done in {:.2} seconds with quality {}'.format(time_elapsed, quality))

In [108]:
run_all()

Solving instance add20.graph… done in 5.4e+01 seconds with quality 13068
Solving instance cti.graph… done in 5.9e+01 seconds with quality 94052
Solving instance t60k.graph… done in 5.3e+01 seconds with quality 178236
Solving instance m14b.graph… done in 5.9e+01 seconds with quality 2902810


## Выводы
На первом и четвертом тесте ответы лучше, чем при локальном поиске, на втором и третьем тесте ответы совпадают с исходными, даже если изменять стартовый набор, скорее всего этот ответ очень близок к оптимальному и не улучшается данной оптимизацией.